# Developmental Stage Influences Gut Microbiome Community Structure and Metabolism in a Hyperandrogenic Mouse Model of Polycystic Ovary Syndrome. Torres et al.

##### Sequence Data for this project was done on two different runs: submitted the same day, but sequenced on different days. Because the same barcoded primer set was used in both runs, we must first demultiplex the reads. Then those resulting (demultiplexed) reads may be merged and filtered based on study.   

#### This analysis only uses the R1 (forward reads) read in order to replicate the strategies and methods used in 'The Gut Microbiome Is Altered in a Letrozole Induced Mouse Model of Polycystic Ovary Syndrome' -Kelley et al (http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0146509) - and compare the results 

### We were not given a barcode.fq file (these are embedded in the raw fastq files). A work around was to first convert our fastq to fataqual on QIIME (http://qiime.org/scripts/convert_fastaqual_fastq.html)#\
#cd /home/skelley/PCOS_QIIME/8_week_mice/

In [ ]:
#Part 1 of sequencing run 6_20_16
!convert_fastaqual_fastq.py -c fastq_to_fastaqual -f /Volumes/PBD/PCOS.PhD/PCOS_illumina_raw_CHptt6_20_16/Kina-pool-1_S1_L001_R1_001.fastq -o /Volumes/PBD/PCOS.PhD/PCOS_not_merged_6_20_16/fastaqual 

In [ ]:
#Part 2 of sequencing run 7_14_16
!convert_fastaqual_fastq.py -c fastq_to_fastaqual -f /Volumes/PBD/PCOS.PhD/PCOS_illumina_raw_CHpt2.7.14.16/Kina-run2_S1_L001_R1_001.fastq -o /Volumes/PBD/PCOS.PhD/PCOS_not_merged_7_14_16/fastaqual_7_14_16   

## Use new fasta file and use split_libraries.py – Split libraries according to barcodes specified in mapping file
```mapping file names are: Run_6_20_16_ T0_T3_mappingfile.txt and Run_714_16_Mapping_file_T3_T5.txt```

In [ ]:
#6-20-16
!split_libraries.py -m Run_6_20_16_ T0_T3_mappingfile.txt -f PCOS_not_merged_6_20_16/fastaqual/Kina-pool-1_S1_L001_R1_001.fna -q PCOS_not_merged_6_20_16/fastaqual/Kina-pool-1_S1_L001_R1_001.qual -o split_libraries_output_6_20_16/
#7-14-16
!split_libraries.py -m Run_714_16_Mapping_file_T3_T5.txt -f PCOS_not_merged_7_14_16/fastaqual/Kina-pool-1_S1_L001_R1_001.fna -q PCOS_not_merged_7_14_16/fastaqual/Kina-pool-1_S1_L001_R1_001.qual -o split_libraries_output_7_14_16/

### Merged resulting fna tables and the two mapping files corresponding to the two sequencing runs

In [ ]:
cat 6_20_16_not_merged/split_libraries_output_6_20_16/seqs.fna 7_14_16_not_merged/split_output_7_14_16/seqs.fna >> merged_fna/merged.fna

# merge two mapping files
!merge_mapping_files.py -m PCOS_not_merged_6_20_16/Run_6_20_16_T0_T3_mappingfile.txt,PCOS_not_merged_7_14_16/Run_714_16_Mapping_file_T3.T5.txt -o merged_mapping.txt -n 'Data not collected'

### Filter out all adult PCOS samples. They are listed the mapping file as 'kina_exp' under 'study'.
#### Manually removed any study listed as 'CH' in the mapping file and only left those labeled 'kina_exp'. Mapping file went from 288 rows to 96. Modified mapping file named: mapping_file_kina.txt

#### Keep samples from the merged fna table that are in mapping_file_kina.txt aka week 8 (adult) mouse model study
#Keep all sequences that show up in an OTU map.(http://qiime.org/scripts/filter_fasta.html)

### Filter out non-Adult PCOS Mice

In [ ]:
# fitler using this command
!filter_fasta.py -f merged_fna/merged.fna -o 8week/merged_sfiltered.fasta -m mapping_file_kina.txt

### Pick DeNovo OTUs

In [ ]:
!pick_de_novo_otus.py -a -O 4 -i 8week/merged_sfiltered.fasta -o otus_8week_de_novo

### Filter OTU table (Same as 'The Gut Microbiome Is Altered in a Letrozole-Induced Mouse Model of Polycystic Ovary Syndrome'  Kelley et al. paper) : OTUs must be found in a minimum of 40 samples 

In [ ]:
!filter_otus_from_otu_table.py -s 40 -i otu_table.biom -o otu_table_filtered40_8week.biom

In [ ]:
# filtered out samples 50 and 53 from further analysis since they did not have an increase in testosterone, which is indicative of letrozole pellet working properly. So added NA to those samples in the mapping file and then filtered it with the following commands
!filter_samples_from_otu_table.py -i otu_table_filtered40_8week.biom -m mapping_file_kina_NA50_53.txt -s 'Treatment:*,!NA' -o otu_table_filtered40_8week_no_50_53.biom --output_mapping_fp mapping_file_kina_dropped_samples.txt

#changed otu_table_filtered40_8week_no_50_53.biom back to otu_table_filtered40_8week.biom
mv otu_table_filtered40_8week_no_50_53.biom otu_table_filtered40_8week.biom

In [ ]:
!biom summarize-table -i otu_table_filtered40_8week.biom -o otu_table_filtered40_8week_summary.txt

"""Num samples: 84 - dropped 50 and 53 from this analyses since they did not have an increase in testosterone
Num observations: 1155
Total count: 4350802
Table density (fraction of non-zero values): 0.721

Counts/sample summary:
 Min: 7155.0
 Max: 129326.0
 Median: 49746.000
 Mean: 51795.262
 Std. dev.: 27946.143
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy
 Lowest sequencing depth is 49.T0: 7155.0"""

### Filter out singletons

In [ ]:
# filter singletons
!filter_otus_from_otu_table.py -i otus_8week_de_novo/otu_table_filtered40_8week.biom -o otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -n 2

### Summarize taxa through plots

In [ ]:
# Summarize based on treatment
!summarize_taxa_through_plots.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -o taxa_summary_treatment -m mapping_file_kina_dropped_samples.txt -c Treatment -s
#summarize based on treatment and time (weeks)
!summarize_taxa_through_plots.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -o taxa_summary_treatmenttime -m mapping_file_kina_dropped_samples.txt -c treatment_time -s


# rarefy taxa to 7000
!single_rarefaction.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -o otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons_e7000.biom -d 7000
# summarize taxa rarefied 7000
!summarize_taxa_through_plots.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons_e7000.biom -o taxa_summary_treatmentE7000 -m mapping_file_kina_dropped_samples.txt -c treatment_time -s -f

### Alpha Diversity ( Full alpha diversity analysis can be found in R folder) 

In [ ]:
!alpha_diversity.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -o alpha_diversity.txt -t otus_8week_de_novo/rep_set.tre 
# add alpha diversity to the mapping file
!add_alpha_to_mapping_file.py -i alpha_diversity.txt -m mapping_file_kina_dropped_samples.txt -o mapping_file_kina_dropped_samples_alphaDiv.tsv
# later added the simpsons diversity analysis and equitability
!alpha_diversity.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -o simpson_equitabilitydiversity.txt -t otus_8week_de_novo/rep_set.tre -m equitability,simpson
 #add to mappig file
!add_alpha_to_mapping_file.py -i simpson_equitabilitydiversity.txt -m mapping_file_kina_dropped_samples.txt -o mapping_file_kina_dropped_samples_alphaDiv.tsv

## Alpha diversity plot using the above data was done in R and will be under the Alpha_diversity folder

### Beta diversity and ANOSIM post-treatment (no time 0) ( Letrozole:36  Placebo  :48  )

In [ ]:
# filter time 0
!filter_samples_from_otu_table.py -i otus_8week_de_novo/otu_table_filtered40_8week_rmsingletons.biom -m mapping_file_kina_dropped_samples.txt -s 'time:*,!0' -o otu_table_filtered40_rmsingletons_postTreatment.biom --output_mapping_fp mapping_file_kina_dropped_samples_postTreatment.txt
#beta diversity
!beta_diversity_through_plots.py -i otu_table_filtered40_rmsingletons_postTreatment.biom -m mapping_file_kina_dropped_samples_postTreatment.txt -t otus_8week_de_novo/rep_set.tre --color_by_all_fields -o beta_div_minus_time0_sd146/ -e 146

"""The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. 
The smallest eigenvalue is -0.0736000912521 and the largest is 1.98347378061."""

In [ ]:
#unweighted
!compare_categories.py --method anosim -i beta_div_minus_time0_sd146/unweighted_unifrac_dm.txt -m mapping_file_kina_dropped_samples_postTreatment.txt -o anosim_unweighted -c Treatment
"""method name	ANOSIM
test statistic name	R
sample size	70
number of groups	2
test statistic	0.08502743484224963
p-value	0.003"""

In [ ]:
#weighted
!compare_categories.py --method anosim -i beta_div_minus_time0_sd146/weighted_unifrac_dm.txt -m mapping_file_kina_dropped_samples_postTreatment.txt -o anosim_weighted -c Treatment

"""method name	ANOSIM
test statistic name	R
sample size	70
number of groups	2
test statistic	0.1913388203017832
p-value	0.001"""

### 2D beta diversity plots for paper

In [ ]:
# unweighted
!make_2d_plots.py -i beta_div_minus_time0_sd146/unweighted_unifrac_pc.txt -m mapping_file_kina_dropped_samples_postTreatment.txt -o 2d_plots_sd146/
#weighted
! make_2d_plots.py -i beta_div_minus_time0_sd146/weighted_unifrac_pc.txt -m mapping_file_kina_dropped_samples_postTreatment.txt -o 2d_plots_sd146_weighted/

### Data processing for the 8 week PCOS mouse model (Adult) was done exactly as instructed in the Kelley et al. PCOS paper (http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0146509). Merging of the 8 week (Adult) biom file with the 4 week (pubertal) biom file obtained from Kelley et al.  Data files were merged solely to look at the phylum, family and class level. We only compared at high taxonomic levels to circumvent both possible dubious results due to the stochastic nature of OTU picking and the potential taxonomic misclassifications due to sequencing bias.

### Merge 4 week (pubertal) and 8 week (adult) Biom and Mapping files

In [ ]:
#merge biom table
!merge_otu_tables.py -i otu_table_filtered_MIN40.biom otu_table_filtered40_8week_rmsingletons.biom -o 4week_8week_merged/merged.biom
# merge mapping files
!merge_mapping_files.py -i mapping_file_kina_dropped_samples.txt,pcos_mapping.txt -o mapping_file_4and8week.txt

### Taxa Plot Summary

In [ ]:
#sumamrize taxa plots based on subject id
summarize_taxa_through_plots.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/merged.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/mapping_file_4and8week.txt -o taxa_plots -s

#summarize taxa plots- grouped by treatment group ( 4 week Placebo, 4 week Letrozole, 8 wekk Placebo, 8 week Letrozole)
!summarize_taxa_through_plots.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/merged.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/mapping_file_4and8week.txt -o taxa_plots_Treatment.study -c Treatment.study -s

#### Group Significance  (this was done at the genus level, L6 file in summarize_taxa, due to too many significant OTUs which made it hard to really see patterns- we thought genus level would help to focus the comparison a bit more). For group significance, heatmap, group significance and randomForest approach, developmental stage samples (aka 8 week vs 4 week) were analyzed separately not as one merged biom file.

### 8 Week (Adult) Analysis 

In [ ]:
# filter out 8 week samples
!filter_samples_from_otu_table.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/merged_sorted_L6_min.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/mapping_file_4and8week.txt -s 'project_name:*,!PCOS Study' -o /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week8analysis/week8.biom --output_mapping_fp /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week8analysis/week8mappingfile.txt
# group significance - kruskal- wallis
!group_significance.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week8analysis/week8.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week8analysis/week8mappingfile.txt -c Treatment_study -o /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week8analysis/kw_week8.txt

### 4 Week (Pubertal) Analysis

In [ ]:
# filter out 4 week samples
!filter_samples_from_otu_table.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/merged_sorted_L6_min.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/mapping_file_4and8week.txt -s 'project_name:*,!no_data' -o /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week4analysis/L6_week4.biom --output_mapping_fp /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week4analysis/mappingfile_week4.txt
# group significance - kuskal - wallis
!group_significance.py -i /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week4analysis/L6_week4.biom -m /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week4analysis/mappingfile_week4.txt -c Treatment_study -o /Users/Pedro_Torres/Desktop/PCOS_Analysis/PCOS.16S.PhD/4weekand8week/taxa_plots/L6_week4analysis/kw_week4.txt

#### Heatmaps and randomForest were done in R studio (v 0.99.893). Both heat map and randomForest are under the 4week_8week_merged folder.